In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from yahooquery import Ticker
import datetime as dt
from deepdow.utils import raw_to_Xy, returns_to_Xy, prices_to_returns
from mlfinlab.portfolio_optimization import RiskEstimators,ReturnsEstimators
from deepdow.data import InRAMDataset
import torch
import mlfinlab

# Create the necessary functions

In [2]:
def get_data(ticker_list, interval = '1d', start = dt.date.today() - dt.timedelta(days=1825), 
             end = dt.date.today()):

    df = Ticker(ticker_list).history(interval = interval, start = start, end = end)[['adjclose','volume']].unstack(level=0)
    df.columns = df.columns.swaplevel(0, 1)
    df.sort_index(axis=1, level=0, inplace=True)
    df.index = df.index.normalize()
    
    return df

In [13]:
def raw_to_xy(raw_data, lookback=10, horizon=10, gap=0, freq='B'):

    asset_names = raw_data.columns.levels[0].to_list()
    indicators = raw_data.columns.levels[1].to_list()

    index = pd.date_range(start=raw_data.index[0], end=raw_data.index[-1], freq=freq)

    new = pd.DataFrame(raw_data, index=index).ffill().bfill()

    absolute = new.iloc[:, new.columns.get_level_values(0).isin(asset_names)][asset_names]
    absolute = absolute.iloc[:, absolute.columns.get_level_values(1).isin(indicators)]

    returns = ReturnsEstimators.calculate_returns(absolute).fillna(0)

    X_list = []
    y_list = []
    for ind in indicators:
        X, timestamps, y = (returns_to_Xy(returns.xs(ind, axis=1, level=1),
                                          lookback=lookback,
                                          horizon=horizon,
                                          gap=gap))
        X_list.append(X)
        y_list.append(y)

    X = np.concatenate(X_list, axis=1)
    y = np.concatenate(y_list, axis=1)

    return X, timestamps, y, asset_names, indicators

# Load Data

In [3]:
ticker_list = ['ORG.AX', 'BEN.AX', 'WOR.AX', 'SYD.AX', 'SUN.AX', 'MQG.AX', 'TAH.AX', 'QBE.AX']
stock_prices = get_data(ticker_list, start = "2016-01-01", end = "2020-06-01")

In [14]:
from deepdow.utils import raw_to_Xy

n_timesteps = len(stock_prices)
n_channels = len(stock_prices.columns.levels[1])
n_assets = len(stock_prices.columns.levels[0])

lookback, gap, horizon = 60, 5, 5

X, timestamps, y, asset_names, indicators = raw_to_xy(stock_prices,
                                                      lookback=lookback,
                                                      gap=gap,
                                                      horizon=horizon)

In [17]:
dataset = InRAMDataset(X, y, timestamps=timestamps, asset_names=asset_names)

In [18]:
from deepdow.data import RigidDataLoader

torch.manual_seed(1)
batch_size = 5

dataloader = RigidDataLoader(dataset, batch_size=batch_size)

# Create Our Networks

Use DeepDow Prebuilt Models: ThorpNet

In [28]:
from deepdow.nn import ThorpNet

n_assets = 8
max_weight = 1
force_symmetric = True
network = ThorpNet(n_assets, max_weight=max_weight, force_symmetric=force_symmetric)

print(network)

ThorpNet(
  (portfolio_opt_layer): NumericalMarkowitz(
    (cvxpylayer): CvxpyLayer()
  )
)
